In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as ssp

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics, model_selection
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import BaggingClassifier
import lightgbm
from lightgbm.sklearn import LGBMClassifier
import os 

/Users/LF/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
work_train = {} 
work_test = {}
pre_process=["w2v","d2v","glove","ftxt","tfidf"]
path="bases/"
for f in pre_process:
    work_train[f] = pd.read_csv(path+"working_train_"+f)
    work_test[f] = pd.read_csv(path+"working_test_"+f)

In [4]:
y=pd.read_csv("bases/training_variants").Class.values - 1

In [18]:
for n in dic_svd:
    for i in range(n):
        X_train['tsvd_' +str(n)+"_"+str(i)] = tsvd_train[n][:, i]
         X_test['tsvd_' +str(n)+"_"+str(i)] = tsvd_test[sv][:, i]

In [19]:
X_train_0=np.array(X_train)
X_test_0=np.array(X_test)

In [28]:
dic_classifier={"XGB_medium":XGBClassifier(max_depth=5, objective="multi:softprob",subsample=0.7,seed=26),
    "XGB_small":XGBClassifier(max_depth=2,objective="multi:softprob",subsample=0.5,seed=26),
                   "XGB_tall":XGBClassifier(max_depth=7,subsample=0.9,objective="multi:softprob",seed=26)
                }

In [23]:
def model_gen(X,X_test,y,classifier,file,five_fold_predict=True):
    #if not os.path.exists("scores/"+file):
    #   os.makedirs("scores/"+file)
    kf = model_selection.StratifiedKFold(n_splits=5, random_state=26, shuffle=True)
    if five_fold_predict:
        fold = 0
        y_test=0
        for train_index, test_index in kf.split(X, y):
        
            fold += 1

            X_train, X_valid    = X[train_index],   X[test_index]
            y_train, y_valid    = y[train_index],   y[test_index]

            print("Fold", fold, X_train.shape, X_valid.shape)

            clf=classifier
            clf.fit(X_train,y_train)
            p_test = clf.predict_proba(X_test)
            y_test += p_test/5
    else:
        print("One Fold predict")
        clf=classifier
        clf.fit(X,y)
        y_test=clf.predict_proba(X_test)
        print("One Fold done")
    classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
    subm = pd.DataFrame(y_test, columns=classes)
    subm['ID'] = ID_test
    
    subm.to_csv("scores/stack_test/{}.csv".format(file),index=False)
    
    print("cross_val sur train ") #peut etre que to array est exclusivement pour les xgb
    
    if os.path.isfile("scores/stack_train/{}.csv".format(file)):
        print("not necessary, already done")
    else:
        y_pred=cross_val_predict(estimator=clf,X=X,y=y,cv=kf,method="predict_proba") 
        subm1 = pd.DataFrame(y_pred, columns=classes)
        subm1['ID'] = ID_train
        subm1.to_csv("scores/stack_train/{}.csv".format(file),index=False)


In [36]:
#for clf in dic_classifier:
#   model_gen(X=X_train_0,X_test=X_test_0,y=y,classifier=dic_classifier[clf],file=clf+"_tsvd")

In [31]:
#Add 3 lightGBM hihi

In [32]:
dic_lightgbm={"LGBM_small" : LGBMClassifier(num_leaves=25,bagging_fraction=0.6,
feature_fraction=0.6,application="multiclass",num_class=9),
"LGBM_mid" : LGBMClassifier(num_leaves=40,bagging_fraction=0.8,feature_fraction=0.8,
        application="multiclass",num_class=9),
"LGBM_tall" : LGBMClassifier(num_leaves=70,num_iterations=150,
        application="multiclass",num_class=9)}

In [37]:
#for clf in dic_lightgbm:
#   model_gen(X=X_train_0,X_test=X_test_0,y=y,classifier=dic_lightgbm[clf],
#            file=clf+"_tsvd")

In [ ]:
#add 3 svm

In [ ]:
dic_svm={"svm_lin":BaggingClassifier(svm.SVC(kernel="linear",probability=True),max_samples=0.1),
        "svm_rbf":BaggingClassifier(svm.SVC(kernel="rbf",probability=True),max_samples=0.1),
        "svm_poly":BaggingClassifier(svm.SVC(kernel="poly",probability=True),max_samples=0.1),
        "svm_sig":BaggingClassifier(svm.SVC(kernel="sigmoid",probability=True),max_samples=0.1)}

In [ ]:
for clf in dic_svm:
    model_gen(X=X_train_0,X_test=X_test_0,y=y,classifier=dic_svm[clf],
             file=clf+"_tsvd",five_fold_predict=False)

One Fold predict


In [21]:
dic_logit={"logit":LogisticRegression(),
          "ridge":LogisticRegression(penalty="l1")}

In [24]:
for clf in dic_logit:
    model_gen(X=X_train_0,X_test=X_test_0,y=y,classifier=dic_logit[clf],
             file=clf+"_tsvd",five_fold_predict=False)

One Fold predict
One Fold done
cross_val sur train 
One Fold predict
One Fold done
cross_val sur train 
